In [1]:
import urllib.request
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd

link = 'https://rally-base.com/2023/rally-guanajuato-mexico-2023/?ssId='
startat, no_ss=7376, int(23) # starting number of url, count of Special Stages
canceled = [15]

stages = [number for number in range(no_ss)]


rally_23 = []

In [2]:
if canceled:
    for j in canceled: stages.remove(j-1)
#print(stages)


In [3]:
for ss in stages:
    val= startat + ss
    ss_a = str(val)
    my_url11 = link + ss_a
    #print(ss, val, ss_a, "\n", my_url11)
    
    req = urllib.request.Request(my_url11, headers={'User-Agent': 'Mozilla/5.0'})
    uClient11 = urllib.request.urlopen(req)
    page_html11 = uClient11.read()
    uClient11.close()
    data = pd.read_html(page_html11)[1]
   
    data.columns = data.iloc[0]
    data = data[1:]
    data['ss']=ss+1
    #print(data.columns)
    
    equal = '-' in data['Pos.'].unique()
    if equal:
        data['Pos.'] = data['Pos.'].replace('-', method='ffill')
    
    data.to_csv('01_rally23_SS'+str(ss+1)+'.csv', index=False)

    rally_23.append(data)

In [4]:
rally2023_stages = pd.concat(rally_23, axis=0)
#rally2023_stages['Pos.'] = rally2023_stages['Pos.'].astype(str).astype(int)
rally2023_stages['No.'] = rally2023_stages['No.'].astype(str).astype(int)
rally2023_stages.to_csv('03_rally2023.csv', index=False)
rally2023_stages = rally2023_stages.fillna("-")
rally2023_temp=rally2023_stages.drop(['Group','SS time', 'Aver. speed sec/km', 'Diff.Leader Diff.Prev.'], axis=1)
rally2023_view = rally2023_temp.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')

rally2023_temp2 = rally2023_temp[rally2023_temp['No.']<30]
rally2023_view2 = rally2023_temp2.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')
rally2023_view = rally2023_view.fillna("-")


In [5]:
pd.set_option("display.max_columns", None)
rally2023_view

0                                                      Pos.                  \
ss                                                       1   2   3   4   5    
No. Driver / Co-driver Vehicle                                                
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...    3   2   1   2   1   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...    6   6   3   4   3   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...    8   6  30  28  26   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...    1   1  29  24  22   
9   Serderidis  J.  /  Miclotte  F.  Ford Puma Rall...   19  17  30  28  26   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...    5   5   5   7   5   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...    6   4   2   1   2   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...    9   8   7   6  26   
20  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   12  12   8  10   9   
21  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...   16  13   8  11   7   
22  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...   20  15   8   9   8   
23  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...   11  11   8  30  29   
24  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...   10  10   8   8  10   
25  Kajetanowicz  K.  /  Szczepaniak  M.  Škoda Fab...   13  14   8  12  11   
26  Prokop  Martin  /  Ernst  Michal  Ford Fiesta R...   21  19   8  14  13   
27  Chwist  Daniel  /  Heller  Kamil  Škoda Fabia R...   25  24   8  21  19   
28  Castro  Eduardo  /  Mussano  F.  Škoda Fabia Ra...   23  18   8  16  14   
29  Martínez  Jorge  /  Álvarez  A.  Škoda Fabia R5      14  16   8  13  12   
30  Dominguez  Diego  /  Peñate  Rogelio  Ford Fies...   24  25   8  20  18   
31  Cordero  Ricardo  /  Hernández  M.  Citroën C3 ...   22  23   8  18  17   
32  Granados  Miguel  /  Martí  Marc  Škoda Fabia R...   18  22   8  16  30   
33  Evans  Elfyn  /  Martin  Scott  Toyota GR Yaris...    4   8   4   3   4   
34  Salas  Carlos  /  Zapata  Jaime  Škoda Fabia R5      16  19   8  15  15   
35  Coyne  John  /  Treacy  Eoin  Ford Fiesta Rally...   26  27   8  23  20   
36  Mauro  Alejandro  /  Sanjuán  Diego  Škoda Fabi...   14  21   8  19  16   
37  Bailey  Jason  /  Peterson  Shayne  Ford Fiesta...   28  26   8  22  21   
38  Spitalier  P.  /  Spitalier  N.  Renault Clio R...   27  29   8  25  23   
39  Urióstegui  G.  /  Zapata  Armando  Renault Cli...   29  30   8  31  31   
41  Maggio  Rafael  /  Pimentel  M.  Renault Clio R...   31  31   8  27  25   
42  Retama  Emiliano  /  Retama  Diego  Renault Cli...   30  28   8  26  24   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...    2   2   6   5   6   

0                                                                           \
ss                                                      6   7   8   9   10   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   1   2   1   1   7   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...   4  13   5   5   1   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...  26  26  25  25  23   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   7  11   8  10   2   
9   Serderidis  J.  /  Miclotte  F.  Ford Puma Rall...  26  26  25  25  23   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   5   4   2   2   3   
17  Ogier  S.  /  Landais  Vincent  Toyota GR Yaris...   2   1   3   4   3   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...  26  26  25  25  23   
20  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   8   7   7   9  10   
21  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...   9   6  19   3   8   
22  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...  10   9   9   8   9   
23  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...   -   -   -   -   -   
24  Lindholm  Emil  /  Hämäläinen  R.  Škoda Fabia ...  11   8  11  12  11   
25  Kajetanowicz  K.  /  Szczepaniak  M.  Škoda Fab...  12  10  1